In [ ]:
"""Create AzureDocument DB, 
    a collection in it, load a few records to that collection,
    and query those records using SQL syntax.
    ENDPOINT= URI of SQL API cosmosDB account
    MASTERKEY=primary key
"""

import pydocumentdb
import pydocumentdb.document_client as document_client

config = { 
    'ENDPOINT': 'https://pydemo.documents.azure.com:443/',
    'MASTERKEY': 'd7D5iheSPybzGvvEuAhsV3ZE1Csq4AOYSNbW3oCcOWBqFW4GTnZhrRnzDPVspABtUNeqkLJjMoFh4BLNXxB7sQ==',
    'DOCUMENTDB_DATABASE': 'pythondemoDB',
    'DOCUMENTDB_COLLECTION': 'testItems'}

client = document_client.DocumentClient(
                config['ENDPOINT'], 
                {'masterKey': config['MASTERKEY']})


In [ ]:
#Create a DB and a collection
db = client.CreateDatabase({ 'id': config['DOCUMENTDB_DATABASE'] })
options = {
        'offerEnableRUPerMinuteThroughput': True,
        'offerVersion': "V2",
        'offerThroughput': 400}
collection = client.CreateCollection(db['_self'], 
                    { 'id': config['DOCUMENTDB_COLLECTION'] }, 
                    options)

In [ ]:
# Create some documents
document1 = client.CreateDocument(collection['_self'],
    { 
        'id': 'server1',
        'Web Site': 0,
        'Cloud Service': 0,
        'Virtual Machine': 0,
        'name': 'some' 
    })

document2 = client.CreateDocument(collection['_self'],
    { 
        'id': 'server2',
        'Web Site': 1,
        'Cloud Service': 0,
        'Virtual Machine': 0,
        'name': 'some' 
    })

In [ ]:
# Query those documents in SQL
query = { 'query': 'SELECT * FROM server s' }    
options = {'enableCrossPartitionQuery': True, 'maxItemCount':2 } 
result_iterable = client.QueryDocuments(
                collection['_self'], query, options)
results = list(result_iterable);
from pprint import pprint as pp
for item in results:
    print("____________________________")
    pp(item)

In [ ]:
#List all databases in your CosmosDB account
for item in list(client.ReadDatabases()):
    print(item['id'])

In [ ]:
#Create one more collection
db_link='dbs/' + config['DOCUMENTDB_DATABASE']

def create_collection(client, db_link, id):
    client.CreateCollection(db_link, {"id": id})
    print('Collection with id \'{0}\' created'.format(id))
create_collection(client, db_link, "dummy")

In [ ]:
 #List all collections in a database 
def listCollectionsInDB(db_link):
    collections = list(client.ReadCollections(db_link))
    if not collections:
        print("no collections here")
    else:
        for collection in collections:
            print(collection['id'])          
listCollectionsInDB(db_link)

In [ ]:
#Delete a collection
def delete_collection(client, db_link, id):
    collection_link = db_link + '/colls/{0}'.format(id)
    client.DeleteCollection(collection_link)
    print('Collection with id \'{0}\' was deleted'.format(id))
delete_collection(client, db_link, "dummy")

In [ ]:
#Delete this database
client.DeleteDatabase(db_link)
print('Database with id \'{0}\' was deleted'.format(config['DOCUMENTDB_DATABASE']))